In [ ]:
import torch
import segmentation_models_pytorch as smp
import cv2
import numpy as np
from scipy.fftpack import dct
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import pandas as pd
from pathlib import Path, PurePath
import os
import time
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.io import gfile
from scipy.spatial.distance import cosine, euclidean
import tensorflow as tf
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model
import librosa
import moviepy.video.io.VideoFileClip as mp

In [ ]:
class UnetEncoder():
  def __init__(self, video_1: str, video_2: str):
    self.transform = transforms.Compose([
        transforms.Resize((256, 256))
    ])

    self.frames1 = self.frames_extract(video_1)
    self.frames2 = self.frames_extract(video_2)
    unet_model = smp.Unet(
      encoder_name="resnet34",
      encoder_weights="imagenet",
      in_channels=3,
      classes=1
    )
    self.encoder = unet_model.encoder


  def frames_extract(self, video_file, sampling_fps=1, max_frame_cnt=60):
    """Извлекает кадры из видео с использованием OpenCV."""
    vcap = cv2.VideoCapture(video_file)
    fps = vcap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps / sampling_fps)
    frames = []

    success, frame = vcap.read()
    count = 0

    while success and count < max_frame_cnt:
        if count % frame_interval == 0:
            frames.append(frame)
        success, frame = vcap.read()
        count += 1

    vcap.release()
    return frames

  def hamming_distance(self, tensor1, tensor2):
    tensor1_flat = tensor1.view(-1, 512)
    tensor2_flat = tensor2.view(-1, 512)

    cosine_sim = F.cosine_similarity(tensor1_flat, tensor2_flat, dim=1)


    cosine_distance = 1 - cosine_sim
    return cosine_distance

  def encode(self):
    frames1 = self.frames1
    frames2 = self.frames2


    similar_frames = 0
    total_frames = 0
    sum = 0
    k = 1
    while True:
        if len(frames1) == k-1 or len(frames2) == k-1:
            break
        frame1 = self.transform(torch.Tensor(frames1[k-1]).permute(2, 0, 1)).unsqueeze(0)
        frame2 = self.transform(torch.Tensor(frames2[k-1]).permute(2, 0, 1)).unsqueeze(0)

        vec1 = self.encoder(frame1)
        vec2 = self.encoder(frame2)
        sdistance = self.hamming_distance(vec1[5], vec2[5])
        k += 1
        sum += sdistance.mean()
    return 1 - float(sum/k)


In [ ]:
class HashEncoder():
  def __init__(self, v1_path, v2_path):
    self.v1_path = v1_path
    self.v2_path = v2_path

  def preprocess_frame(self, frame, size=32):
      gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      resized = cv2.resize(gray, (size, size))
      return resized


  def compute_dct(self, image):
      return dct(dct(image.T, norm='ortho').T, norm='ortho')


  def create_hash(self, dct_block, hash_size=8):
      dct_low = dct_block[:hash_size, :hash_size]
      med = np.median(dct_low)
      return (dct_low > med).flatten()


  def compute_frame_hash(self, frame):
      preprocessed = self.preprocess_frame(frame)
      dct_result = self.compute_dct(preprocessed)
      return self.create_hash(dct_result)


  def hamming_distance(self, hash1, hash2):
      return np.sum(hash1 != hash2)


  def extract_frames(self, video_path, num_frames_per_sec=1):
      cap = cv2.VideoCapture(video_path)
      frames = []
      total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

      fps = cap.get(cv2.CAP_PROP_FPS)
      duration_seconds = int(total_frames // fps)
      indices = np.linspace(0, total_frames - 1, num_frames_per_sec * duration_seconds, dtype=int)
      # print(len(indices))
      for i in indices:
          cap.set(cv2.CAP_PROP_POS_FRAMES, i)
          ret, frame = cap.read()
          if ret:
              frames.append(frame)

      cap.release()
      return frames


  def compute_similarity_matrix(self, frames1, frames2, threshold=0.8):
      if len(frames1) > len(frames2):
          frames1, frames2 = frames2, frames1

      hashes1 = [self.compute_frame_hash(frame) for frame in frames1]
      hashes2 = [self.compute_frame_hash(frame) for frame in frames2]

      similarity_matrix = np.zeros((len(hashes1), len(hashes2)))
      similar_frames = 0
      for i, hash1 in enumerate(hashes1):
          count_in_row = 0
          for j, hash2 in enumerate(hashes2):
              distance = self.hamming_distance(hash1, hash2)
              similarity = 1 - (distance / len(hash1))  # Нормализация (64 бита в хеше)
              if similarity > threshold:
                  count_in_row += 1
              if count_in_row > 0:
                  similar_frames += 1
                  break

              similarity_matrix[i, j] = similarity

      similarity_score = similar_frames / similarity_matrix.shape[0]

      return similarity_matrix, similarity_score


  def compare_videos(self, video1_path, video2_path, num_frames=3):
      frames1 = self.extract_frames(video1_path, num_frames)
      frames2 = self.extract_frames(video2_path, num_frames)

      similarity_matrix, similarity_score = self.compute_similarity_matrix(frames1, frames2)

      return similarity_matrix, similarity_score

  def encode(self):
      if not os.path.exists(self.v1_path) or not os.path.exists(self.v2_path):
          return 0

      t1 = time.time()
      similarity_matrix, similarity_score = self.compare_videos(self.v1_path, self.v2_path)
      return similarity_score


In [ ]:
class ResNetEncoder():
  def __init__(self, video1_path, video2_path):
    self.model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    self.video1_path = video1_path
    self.video2_path = video2_path

  # 1. Извлечение кадров из видео
  def extract_frames(self, video_path, max_frames=10):
      cap = cv2.VideoCapture(video_path)
      frames = []
      success, frame = cap.read()
      count = 0
      while success and count < max_frames:
          frames.append(frame)
          success, frame = cap.read()
          count += 1
      cap.release()
      return frames

  # 2. Извлечение гистограммы цветов
  def extract_color_histogram(self, frame):
      hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
      hist = cv2.calcHist([hsv_frame], [0, 1, 2], None, [50, 60, 70], [0, 180, 0, 256, 0, 256])
      hist = cv2.normalize(hist, hist).flatten()
      return hist

  # 3. Извлечение признаков с помощью предобученной модели ResNet50
  def extract_deep_features(self, frame):
      frame_resized = cv2.resize(frame, (224, 224))
      frame_preprocessed = preprocess_input(np.expand_dims(frame_resized, axis=0))
      features = self.model.predict(frame_preprocessed, verbose=0)
      return features.flatten()

  # 5. Комбинирование всех признаков (гистограмма цветов, ResNet, ORB)
  def extract_combined_features(self, frame):
      color_hist = self.extract_color_histogram(frame)
      deep_features = self.extract_deep_features(frame)

      # Объединение всех признаков в один вектор
      combined_features = np.hstack((color_hist, deep_features))
      return combined_features

  # 6. Извлечение кадров из видео и получение средних признаков
  def extract_video_features(self, video_path, max_frames=10):
      frames = self.extract_frames(video_path, max_frames)
      combined_features_list = [self.extract_combined_features(frame) for frame in frames]

      # Усредняем признаки по кадрам
      average_features = np.mean(combined_features_list, axis=0)
      return average_features

  # 7. Сравнение двух видео
  def compare_videos(self, video_path1, video_path2, max_frames=10):
      features_video1 = self.extract_video_features(video_path1, max_frames)
      features_video2 = self.extract_video_features(video_path2, max_frames)

      # Сравнение признаков с использованием косинусного расстояния
      similarity = 1 - cosine(features_video1, features_video2)
      return similarity

  def encode(self):
    similarity_score = self.compare_videos(self.video1_path, self.video2_path)
    return similarity_score

In [ ]:
# Загрузка предобученной модели Inception-v3
model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, pooling='avg')

# Сохранение модели в формате Keras
model.save("inception_v3_model.keras")

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
class InceptionEncoder():
  def __init__(self, video1_path, video2_path=None):
    self.video1_path = video1_path
    self.video2_path = video2_path

  def frames_extract(self, video_file, start_off=0, sampling_fps=1, max_frame_cnt=60):
      """Extracts frames from input video."""
      if not os.path.exists(video_file):
          return None

      vcap = cv2.VideoCapture(video_file)
      fps = vcap.get(cv2.CAP_PROP_FPS)
      frame_interval = int(fps / sampling_fps) if sampling_fps > 0 and sampling_fps < fps else 1
      frame_list, cnt = [], 0

      success, im = vcap.read()
      while success and cnt < max_frame_cnt:
          if cnt % frame_interval == 0:
              frame_list.append(im)
          cnt += 1
          success, im = vcap.read()

      return np.array(frame_list) if frame_list else None

  def create_graph(self, saved_model_dir):
      """Load the Inception-v3 model from a SavedModel directory."""
      model = tf.saved_model.load(saved_model_dir)
      return model

  def feature_from_single_image_file(self, image_file, model):
      """Extract feature vector from a single image file."""
      if not gfile.exists(image_file):
          print(f"File does not exist: {image_file}")
          return None

      image_data = gfile.GFile(image_file, 'rb').read()
      image_tensor = tf.image.decode_jpeg(image_data, channels=3)
      image_tensor = tf.image.resize(image_tensor, (299, 299))  # Inception-v3 input size

      image_tensor = tf.expand_dims(image_tensor, axis=0)
      image_tensor = image_tensor / 255.0

      print(model.signatures['serving_default'](image_tensor))
      feature_tensor = model.signatures['serving_default'](image_tensor)['pool_3']
      return np.squeeze(feature_tensor.numpy())

  def feature_from_single_video_file(self, video_file, model=tf.keras.models.load_model("inception_v3_model.keras"), start_off=0, sampling_fps=1, max_frame_cnt=60, padding=True):
      """Extract feature vectors from a video file."""
      if not gfile.exists(video_file):
          print(f"File does not exist: {video_file}")
          return None

      frames = self.frames_extract(video_file, start_off, sampling_fps, max_frame_cnt)
      if frames is None:
          return None

      features = []
      for frame in frames:
          frame_tensor = tf.convert_to_tensor(frame, dtype=tf.uint8)
          frame_tensor = tf.image.resize(frame_tensor, (299, 299))  # Inception-v3 input size
          frame_tensor = tf.expand_dims(frame_tensor, axis=0)
          frame_tensor = frame_tensor / 255.0

          # Получаем эмбеддинги напрямую через вызов модели
          feature_tensor = model(frame_tensor)
          features.append(np.squeeze(feature_tensor.numpy()))

      # Padding if necessary
      if padding and max_frame_cnt > len(features):
          zero_feat = np.zeros([2048], dtype=np.float32)
          features.extend([zero_feat] * (max_frame_cnt - len(features)))

      return np.array(features)

  def encode(self, model):
    # Загрузка модели в формате Keras
    model = self.model
    if video_features_2:
        # Теперь можно использовать модель напрямую
        video_features_1 = self.feature_from_single_video_file(self.video1_path , model, sampling_fps=1, max_frame_cnt=60, padding=True)
        video_features_2 = self.feature_from_single_video_file(self.video2_path , model, sampling_fps=1, max_frame_cnt=60, padding=True)

        from sklearn.metrics.pairwise import cosine_similarity

        # Получаем средние эмбеддинги по кадрам для каждого видео
        mean_features_1 = np.mean(video_features_1, axis=0)
        mean_features_2 = np.mean(video_features_2, axis=0)

        # Рассчитываем косинусное сходство
        similarity = cosine_similarity([mean_features_1], [mean_features_2])[0][0]
        return similarity
    else:
        print('Нет 2-го видео')


In [ ]:

class MovieNetEncoder():
  def __init__(self, video_file_1 ,video_file_2):
    self.video_file_1 = video_file_1
    self.video_file_2 = video_file_2
    self.model_id = 'a0'
    self.resolution = 224
    self.movinet_model = movinet.Movinet(model_id=self.model_id)
    self.movinet_model.trainable = False

  def frames_extract(self, video_file, sampling_fps=1, max_frame_cnt=60):
      """Извлекает кадры из видео с использованием OpenCV."""
      vcap = cv2.VideoCapture(video_file)
      fps = vcap.get(cv2.CAP_PROP_FPS)
      frame_interval = int(fps / sampling_fps)
      frames = []

      success, frame = vcap.read()
      count = 0

      while success and count < max_frame_cnt:
          if count % frame_interval == 0:
              frame_resized = cv2.resize(frame, (224, 224))  # Изменение размера кадра под вход модели
              frames.append(frame_resized)
          success, frame = vcap.read()
          count += 1

      vcap.release()
      return np.array(frames)

  def prepare_video(self, frames):
      """Подготовка кадров видео для подачи в MoViNet."""
      frames = np.array(frames, dtype=np.float32)
      frames = frames / 255.0  # Нормализация пикселей
      frames = np.expand_dims(frames, axis=0)  # Добавляем batch размерность
      return frames

  def extract_movinet_features(self, video_tensor, model):
      """Извлекает эмбеддинги из видео с помощью модели MoViNet (feature extractor)."""
      output = model(tf.convert_to_tensor(video_tensor))  # Применение модели
      features = output[0]['head']  # Доступ к признакам
      return np.squeeze(features.numpy())  # Преобразование в numpy

  def embed(self, video_file, sampling_fps=1, max_frame_cnt=60):
      frames = self.frames_extract(video_file)
      video_tensor = self.prepare_video(frames)
      video_features = self.extract_movinet_features(video_tensor, self.movinet_model)
      return video_features


  def compare_videos(self, video_path1, video_path2, max_frames=10):
      features_video1 = self.embed(video_path1)
      features_video2 = self.embed(video_path2)
      cos_similarity = 1 - cosine(features_video1, features_video2)
      euclidean_distance = euclidean(features_video1, features_video2)

      return cos_similarity, euclidean_distance

  def encode(self, sampling_fps=1, max_frame_cnt=60):
      cos_sim, euclid_dist = self.compare_videos(self.video_file_1, self.video_file_2)

      return euclid_dist

In [ ]:
class AudioEncoder():
  def __init__(self, video_file_1 ,video_file_2):
    self.video_file_1 = video_file_1
    self.video_file_2 = video_file_2

  def make_specrtum(self, video):
    video1 = mp.VideoFileClip(video)
    video1.audio.write_audiofile("audio1.wav")

    y1, sr1 = librosa.load("audio1.wav", sr=None)
    S1 = librosa.feature.melspectrogram(y=y1, sr=sr1)
    mean_spectrum1 = np.mean(S1, axis=1)
    return mean_spectrum1

  def encode(self):
    similarity = 1 - cosine(self.make_specrtum(self.video_file_1), self.make_specrtum(self.video_file_2))
    return similarity

In [ ]:
df = pd.read_csv('test.csv')
df.head()

,Unnamed: 0,created,uuid,link
0,0,2024-06-02 07:19:29,199d5f08-49ae-4f9a-8fc6-52b251c4e958,https://s3.ritm.media/yappy-db-duplicates/199d...
1,1,2024-06-01 14:03:21,283e4172-e617-4726-8ded-adb9d6ba0b37,https://s3.ritm.media/yappy-db-duplicates/283e...
2,2,2024-06-14 13:05:33,918a9b20-38df-4079-98d3-29fef669c3d2,https://s3.ritm.media/yappy-db-duplicates/918a...
3,3,2024-06-03 13:24:34,83160552-d535-4c1a-9d61-aa7422df6ff8,https://s3.ritm.media/yappy-db-duplicates/8316...
4,4,2024-06-02 07:41:49,4f59f7c9-4091-4565-bbbd-16ed3ba51cad,https://s3.ritm.media/yappy-db-duplicates/4f59...


In [ ]:
import requests

def download_video(video_url):
    # Get the response from the video URL
    response = requests.get(video_url, stream=True)

    # Extract the filename from the URL
    filename = video_url.split("/")[-1]

    # Open a local file for writing the downloaded content
    with open(filename, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

    print(f"{filename} downloaded!")


# download_video(video_link)

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
for video_link in tqdm(df['link'].unique()[250:]):
    download_video(video_link)


In [ ]:
# Загрузка предобученной модели Inception-v3
model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, pooling='avg')

# Сохранение модели в формате Keras
model.save("inception_v3_model.keras")

# Загрузка модели в формате Keras
inception_v3_model = tf.keras.models.load_model("inception_v3_model.keras")


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.io import gfile

def frames_extract(video_file, start_off=0, sampling_fps=1, max_frame_cnt=60):
	"""Extracts frames from input video."""
	if not os.path.exists(video_file):
		return None

	vcap = cv2.VideoCapture(video_file)
	fps = vcap.get(cv2.CAP_PROP_FPS)
	frame_interval = int(fps / sampling_fps) if sampling_fps > 0 and sampling_fps < fps else 1
	frame_list, cnt = [], 0

	success, im = vcap.read()
	while success and cnt < max_frame_cnt:
		if cnt % frame_interval == 0:
			frame_list.append(im)
		cnt += 1
		success, im = vcap.read()

	return np.array(frame_list) if frame_list else None

def create_graph(saved_model_dir):
	"""Load the Inception-v3 model from a SavedModel directory."""
	model = tf.saved_model.load(saved_model_dir)
	return model

def feature_from_single_image_file(image_file, model):
	"""Extract feature vector from a single image file."""
	if not gfile.exists(image_file):
		print(f"File does not exist: {image_file}")
		return None

	image_data = gfile.GFile(image_file, 'rb').read()
	image_tensor = tf.image.decode_jpeg(image_data, channels=3)
	image_tensor = tf.image.resize(image_tensor, (299, 299))  # Inception-v3 input size

	image_tensor = tf.expand_dims(image_tensor, axis=0)
	image_tensor = image_tensor / 255.0

	feature_tensor = model.signatures['serving_default'](image_tensor)['pool_3']
	return np.squeeze(feature_tensor.numpy())

def feature_from_single_video_file(video_file, model=inception_v3_model, start_off=0, sampling_fps=1, max_frame_cnt=60, padding=True):
	"""Extract feature vectors from a video file."""
	if not gfile.exists(video_file):
		print(f"File does not exist: {video_file}")
		return None

	sampling_fps = sampling_fps
	max_frame_cnt = max_frame_cnt
	start_off = start_off
	frames = frames_extract(video_file, start_off, sampling_fps, max_frame_cnt)
	if frames is None:
		return None

	features = []
	for frame in frames:
		frame_tensor = tf.convert_to_tensor(frame, dtype=tf.uint8)
		frame_tensor = tf.image.resize(frame_tensor, (299, 299))  # Inception-v3 input size
		frame_tensor = tf.expand_dims(frame_tensor, axis=0)
		frame_tensor = frame_tensor / 255.0

		# Получаем эмбеддинги напрямую через вызов модели
		feature_tensor = model(frame_tensor)
		features.append(np.squeeze(feature_tensor.numpy()))

	# Padding if necessary
	if padding and max_frame_cnt > len(features):
		zero_feat = np.zeros([2048], dtype=np.float32)
		features.extend([zero_feat] * (max_frame_cnt - len(features)))

	# Усреднение эмбеддингов по фреймам
	averaged_features = np.mean(features, axis=0)

	return averaged_features



In [ ]:
df['link_vid'] = df['uuid'].apply(lambda x: f'/content/{x}.mp4')
df['embeddings'] = df['link_vid'].progress_apply(feature_from_single_video_file)

100%|██████████| 1494/1494 [42:43<00:00,  1.72s/it]


In [ ]:
df.to_csv('sample_data/embeds1500.csv')

In [ ]:
train = pd.read_csv('train.csv')
train

,created,uuid,link,is_duplicate,duplicate_for,is_hard
0,2024-06-01 00:05:43,23fac2f2-7f00-48cb-b3ac-aac8caa3b6b4,https://s3.ritm.media/yappy-db-duplicates/23fa...,False,NaN,False
1,2024-06-01 00:11:01,2fa37210-3c25-4a87-88f2-1242c2c8a699,https://s3.ritm.media/yappy-db-duplicates/2fa3...,False,NaN,False
2,2024-06-01 00:13:20,31cc33d5-95de-4799-ad01-87c8498d1bde,https://s3.ritm.media/yappy-db-duplicates/31cc...,False,NaN,False
3,2024-06-01 00:27:23,03abd0ec-609e-4eea-9f2a-b6b7442bc881,https://s3.ritm.media/yappy-db-duplicates/03ab...,False,NaN,False
4,2024-06-01 00:30:23,22ee0045-004b-4c7e-98f2-77e5e02e2f15,https://s3.ritm.media/yappy-db-duplicates/22ee...,False,NaN,False
...,...,...,...,...,...,...
22758,2024-09-12 13:46:57,0efe756a-e965-40c1-94db-de7f3e6649a9,https://s3.ritm.media/yappy-db-duplicates/0efe...,True,131846f3-6f5c-497a-a2fa-95cfb3929301,False
22759,2024-09-12 14:46:13,caec3b94-e356-4576-b00a-515e0df1dfc3,https://s3.ritm.media/yappy-db-duplicates/caec...,True,3b5eb15a-c6d7-4214-8dd6-c029564ff11d,False
22760,2024-09-13 09:08:42,c5b69151-f240-4e27-a5c9-c41f79a167e9,https://s3.ritm.media/yappy-db-duplicates/c5b6...,True,17ecc94a-f28a-40d5-b438-86b6e82a2fef,False
22761,2024-09-13 14:52:21,6d3233b6-f8de-49ba-8697-bb30dbf825f7,https://s3.ritm.media/yappy-db-duplicates/6d32...,True,1838f7a7-ef2a-4141-a125-90fb5bf0c5a2,False


In [ ]:
full_df = df.merge(train[['uuid', 'is_duplicate', 'duplicate_for']], on='uuid', how='left')

In [ ]:
full_df.rename(columns={'uuid': 'video_1', 'duplicate_for': 'video_2'}, inplace=True)

In [ ]:
full_df['is_duplicate'] = full_df['is_duplicate'].astype(int)

In [ ]:
videos = list(full_df['video_1'])

In [ ]:
full_df.loc[full_df['video_2'].isna(), 'video_2'].index

Index([   0,    1,    3,    4,    5,    6,    7,    8,    9,   10,
       ...
       1473, 1474, 1475, 1476, 1477, 1478, 1479, 1481, 1482, 1485],
      dtype='int64', length=986)

In [ ]:
import random

full_df.loc[full_df['video_2'].isna(), 'video_2'] = full_df.apply(lambda _: random.choice(videos), axis=1)

In [ ]:
full_df.head()

,Unnamed: 0,created,video_1,link,link_vid,embeddings,is_duplicate,video_2
0,0,2024-06-02 07:19:29,199d5f08-49ae-4f9a-8fc6-52b251c4e958,https://s3.ritm.media/yappy-db-duplicates/199d...,/content/199d5f08-49ae-4f9a-8fc6-52b251c4e958.mp4,"[0.030041222, 0.0035602425, 0.019737726, 0.005...",0,e7a4f007-250d-4e58-89b8-4b0166dd9076
1,1,2024-06-01 14:03:21,283e4172-e617-4726-8ded-adb9d6ba0b37,https://s3.ritm.media/yappy-db-duplicates/283e...,/content/283e4172-e617-4726-8ded-adb9d6ba0b37.mp4,"[0.021256344, 0.022595055, 0.004649999, 0.0108...",0,3b447013-9202-415d-ae15-b1d76ce8e060
2,2,2024-06-14 13:05:33,918a9b20-38df-4079-98d3-29fef669c3d2,https://s3.ritm.media/yappy-db-duplicates/918a...,/content/918a9b20-38df-4079-98d3-29fef669c3d2.mp4,"[0.0039061562, 0.015314342, 0.010354506, 0.023...",1,3b48ffa3-db42-4d8c-8f37-6f5cbe2b406c
4,4,2024-06-02 07:41:49,4f59f7c9-4091-4565-bbbd-16ed3ba51cad,https://s3.ritm.media/yappy-db-duplicates/4f59...,/content/4f59f7c9-4091-4565-bbbd-16ed3ba51cad.mp4,"[0.012350063, 0.00752949, 0.007962728, 0.01092...",0,0d6ea6bd-2997-402c-9ecc-286c479148e6
5,5,2024-06-02 17:48:57,2064eb76-9ff1-445a-8eae-2b47e757c9aa,https://s3.ritm.media/yappy-db-duplicates/2064...,/content/2064eb76-9ff1-445a-8eae-2b47e757c9aa.mp4,"[0.01342243, 0.0030780951, 0.025742115, 0.0194...",0,55d1894a-4052-4e77-9769-f2511d3bd786


In [ ]:
full_df.drop(index=[3], inplace=True)

In [ ]:
# full_df.loc[full_df['video_1'] == '55d1894a-4052-4e77-9769-f2511d3bd786', 'embeddings'].values
'55d1894a-4052-4e77-9769-f2511d3bd786' in list(full_df['video_1'])

True

In [ ]:
def create_emb(uuid):
    return full_df.loc[full_df['video_1'] == uuid, ['embeddings']].values[0][0] if uuid in list(full_df['video_1']) else None

In [ ]:
full_df['embendigs_2'] = full_df['video_2'].apply(create_emb)


In [ ]:
full_df.rename(columns={'embeddings': 'embeddings_1', 'embendigs_2': 'embeddings_2'}, inplace=True)

In [ ]:
full_df.drop(index=full_df[full_df['embeddings_2'].isna()].index, inplace=True)

In [ ]:
from scipy.spatial.distance import cosine, euclidean

full_df['Inception_cosin'] = full_df.apply(lambda row: 1 - cosine(row['embeddings_1'], row['embeddings_2']), axis=1)
full_df['Inception_eucld'] = full_df.apply(lambda row: euclidean(row['embeddings_1'], row['embeddings_2']), axis=1)

,Unnamed: 0,created,video_1,link,link_vid,embeddings_1,is_duplicate,video_2,embeddings_2,Inception_cosin,Inception_eucld
0,0,2024-06-02 07:19:29,199d5f08-49ae-4f9a-8fc6-52b251c4e958,https://s3.ritm.media/yappy-db-duplicates/199d...,/content/199d5f08-49ae-4f9a-8fc6-52b251c4e958.mp4,"[0.030041222, 0.0035602425, 0.019737726, 0.005...",0,e7a4f007-250d-4e58-89b8-4b0166dd9076,"[0.026943024, 0.018945308, 0.010458821, 0.0103...",0.621080,0.685089
1,1,2024-06-01 14:03:21,283e4172-e617-4726-8ded-adb9d6ba0b37,https://s3.ritm.media/yappy-db-duplicates/283e...,/content/283e4172-e617-4726-8ded-adb9d6ba0b37.mp4,"[0.021256344, 0.022595055, 0.004649999, 0.0108...",0,3b447013-9202-415d-ae15-b1d76ce8e060,"[0.015870867, 0.009490551, 0.009567837, 0.0124...",0.676761,1.002035
2,2,2024-06-14 13:05:33,918a9b20-38df-4079-98d3-29fef669c3d2,https://s3.ritm.media/yappy-db-duplicates/918a...,/content/918a9b20-38df-4079-98d3-29fef669c3d2.mp4,"[0.0039061562, 0.015314342, 0.010354506, 0.023...",1,3b48ffa3-db42-4d8c-8f37-6f5cbe2b406c,"[0.014696343, 0.018140296, 0.019144671, 0.0087...",0.775978,0.442339
4,4,2024-06-02 07:41:49,4f59f7c9-4091-4565-bbbd-16ed3ba51cad,https://s3.ritm.media/yappy-db-duplicates/4f59...,/content/4f59f7c9-4091-4565-bbbd-16ed3ba51cad.mp4,"[0.012350063, 0.00752949, 0.007962728, 0.01092...",0,0d6ea6bd-2997-402c-9ecc-286c479148e6,"[0.005839845, 0.010342051, 0.02036907, 0.01407...",0.657700,0.594769
5,5,2024-06-02 17:48:57,2064eb76-9ff1-445a-8eae-2b47e757c9aa,https://s3.ritm.media/yappy-db-duplicates/2064...,/content/2064eb76-9ff1-445a-8eae-2b47e757c9aa.mp4,"[0.01342243, 0.0030780951, 0.025742115, 0.0194...",0,55d1894a-4052-4e77-9769-f2511d3bd786,"[0.00015508606, 0.006940969, 0.00827432, 0.006...",0.581669,0.718481


In [ ]:
full_df.loc[~(full_df['is_duplicate'] == 1), 'UNet'].mean()

0.6022722208112095

In [ ]:
full_df['link_vid_2'] = full_df['video_2'].apply(lambda x: f'/content/{x}.mp4')

In [ ]:
def calculate_similarity(row):
    encoder = UnetEncoder(row['link_vid'], row['link_vid_2'])  # Создаем экземпляр для каждой пары видео
    return encoder.encode()

# Создаем новый столбец, используя apply для каждой строки
full_df['UNet'] = full_df.progress_apply(calculate_similarity, axis=1)


  0%|          | 0/1486 [00:00<?, ?it/s]Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth

  0%|          | 0.00/83.3M [00:00<?, ?B/s]
  1%|          | 640k/83.3M [00:00<00:15, 5.66MB/s]
  2%|▏         | 1.38M/83.3M [00:00<00:13, 6.45MB/s]
  3%|▎         | 2.12M/83.3M [00:00<00:12, 6.75MB/s]
  3%|▎         | 2.88M/83.3M [00:00<00:12, 7.01MB/s]
  4%|▍         | 3.62M/83.3M [00:00<00:11, 7.28MB/s]
  5%|▌         | 4.38M/83.3M [00:00<00:11, 7.45MB/s]
  6%|▌         | 5.12M/83.3M [00:00<00:10, 7.54MB/s]
  7%|▋         | 6.00M/83.3M [00:00<00:10, 7.91MB/s]
  8%|▊         | 7.00M/83.3M [00:00<00:09, 8.55MB/s]
 10%|▉         | 8.12M/83.3M [00:01<00:08, 9.32MB/s]
 11%|█▏        | 9.38M/83.3M [00:01<00:07, 10.2MB/s]
 13%|█▎        | 10.6M/83.3M [00:01<00:06, 11.0MB/s]
 14%|█▍        | 12.0M/83.3M [00:01<00:06, 11.8MB/s]
 16%|█▌        | 13.5M/83.3M [00:01<00:05, 12.9MB/s]
 18%|█▊        | 15.1M/83.3M [00:01<00:05

In [ ]:
full_df.to_csv('sample_data/UNet.csv')

In [ ]:
def calculate_audio(row):
    try:
        encoder = AudioEncoder(row['link_vid'], row['link_vid_2'])  # Создаем экземпляр для каждой пары видео
        return encoder.encode()
    except:
        print('ОШИБКА!!!')
        return None

# Создаем новый столбец, используя apply для каждой строки
full_df['audio'] = full_df.progress_apply(calculate_audio, axis=1)


  0%|          | 1/1486 [00:00<03:05,  7.99it/s]

MoviePy - Writing audio in audio1.wav



  0%|          | 1/1486 [00:00<06:04,  4.07it/s]

MoviePy - Done.


  0%|          | 1/1486 [00:00<11:05,  2.23it/s]

...


 56%|█████▋    | 838/1486 [23:19<12:28,  1.16s/it]

MoviePy - Done.


 56%|█████▋    | 839/1486 [23:20<18:55,  1.76s/it]

MoviePy - Writing audio in audio1.wav



chunk:  62%|██████▏   | 672/1078 [00:00<00:00, 1605.31it/s, now=None]


In [ ]:
# Функция для получения длины видео в секундах
def get_video_length(video_file):
    cap = cv2.VideoCapture(video_file)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Количество кадров в секунду
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Общее количество кадров
    cap.release()
    if fps > 0:  # Если fps корректный, считаем длину
        video_length = frame_count / fps  # Длина видео в секундах
    else:
        video_length = 0  # Если fps не найден, длина видео = 0
    return video_length

# Функция для расчета длины видео и их соотношения
def calculate_length_ratio(row):
    length_1 = get_video_length(row['link_vid'])
    length_2 = get_video_length(row['link_vid_2'])

    length_diff = abs(length_1 - length_2)
    max_length = max(length_1, length_2)

    # Избегаем деления на 0
    if max_length > 0:
        length_ratio = length_diff / max_length
    else:
        length_ratio = 0

    return length_ratio

full_df['length_ratio'] = df.apply(calculate_length_ratio, axis=1)


In [ ]:
full_df.to_csv('data.csv')